In [ ]:
# Current directory
import os
os.chdir('E:\Experiment\work')   #   改变当前工作目录到指定路径

### 导入数据

In [13]:
import numpy as np
import pandas as pd

X_new=pd.read_csv(r'G:\\lhy\work\\train_feature\\feature\\AAC.csv',header=None)
y_new= pd.read_csv(r'G:\\lhy\work\data/label_train.csv',header=None)

X_new1=pd.read_csv(r"G:\\lhy\work\\test_feature\\AAC.csv",header=None)
y_new1= pd.read_csv(r'G:\lhy\work\data/label_test.csv',header=None)


print(X_new.shape)
print(y_new.shape)
print(X_new1.shape)
print(y_new1.shape)

X_new = np.array(X_new)
y_new = np.array(y_new).ravel()

# X_new1 = np.array(X_new1)
# y_new1 = np.array(y_new1).ravel()


(2872, 20)
(2872, 1)
(342, 20)
(342, 1)


In [ ]:
from sklearn.preprocessing import StandardScaler

# 使用 StandardScaler 进行标准化
# X_new = X_new[:, 1:]
scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)

### 寻找最佳参数

In [ ]:
import statistics       # 提供数学统计的函数和工具，如平均数、中位数、众数
from sklearn.metrics import balanced_accuracy_score

# dataset splitting   将数据集分割成几个较小的独立的部分，包括训练集、验证集、测试集，保证模型在不同的数据子集上都有良好的表现
from sklearn.model_selection import train_test_split
#   20%的数据作为测试集，80%作为训练集
X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X_new, y_new, test_size=0.2, random_state=1111)

print(X_train_whole.shape)
print(X_ind_test.shape)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


# 定义参数范围
param_grid = {'C': np.logspace(-4,4, num=100), 'max_iter': [5000]}

# 初始化逻辑回归模型
clf = LogisticRegression()

# 初始化网格搜索
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='roc_auc')

# 在训练数据上进行网格搜索
grid_search.fit(X_new, y_new)

# 输出最优参数组合
print("Best parameters:", grid_search.best_params_)

# 输出最优参数组合下的平均值和标准差
print("Best mean balanced accuracy:", round(grid_search.best_score_, 3))
print("Standard deviation of balanced accuracy:", round(grid_search.cv_results_['std_test_score'][grid_search.best_index_], 3))


### 测试

### 10折交叉验证

In [14]:
import numpy as np
import pandas as pd
import math
import statistics
import matplotlib.pyplot as plt  # 导入matplotlib用于绘图
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, log_loss
from sklearn.metrics import f1_score, average_precision_score
from sklearn.metrics import (
    log_loss, confusion_matrix, roc_auc_score, average_precision_score,
    f1_score, precision_recall_curve, auc
)

X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X_new, y_new, test_size=0.2, random_state=1111)

BACC_collecton = []
ACC_collection = []
Sn_collection = []
Sp_collection = []
MCC_collection = []
AUC_collection = []
AP=[]
AP_collection = []
F1_collection = []
AUPR_collection = []

y_true_all = []
y_pred_all = []
y_pred_proba_all = []

mean_recall = np.linspace(0, 1, 100)        
all_precision = []      

base_fpr = np.linspace(0, 1, 100)
mean_tpr = 0.0         

interp_tpr_collection = []  

def categorical_probas_to_classes(p):      
    return np.argmax(p, axis=1)       

# Initialize model
# 138.3209780541621
clf = LogisticRegression(C=138.48863713938746 , max_iter=5000)  
skf = StratifiedKFold(n_splits=5)

# Lists to store losses for plotting
train_losses = []
valid_losses = []

for train, test in skf.split(X_new, y_new):
    # 数据集划分
    X_train, X_valid = np.take(X_new, train, axis=0), np.take(X_new, test, axis=0)
    y_train, y_valid = np.take(y_new, train, axis=0), np.take(y_new, test, axis=0)

    clf.fit(X_train, y_train)

    # Train loss
    y_train_pred_proba = clf.predict_proba(X_train)
    train_loss = log_loss(y_train, y_train_pred_proba)
    train_losses.append(train_loss)

    # Valid loss
    y_valid_pred_proba = clf.predict_proba(X_valid)
    valid_loss = log_loss(y_valid, y_valid_pred_proba)
    valid_losses.append(valid_loss)

    # Prediction
    y_valid_pred = categorical_probas_to_classes(y_valid_pred_proba)

    y_true_all.extend(y_valid.ravel().astype(int))
    y_pred_all.extend(y_valid_pred.astype(int))
    y_pred_proba_all.extend(y_valid_pred_proba[:, 1])

    # Confusion matrix metrics
    TP, FP, FN, TN = confusion_matrix(y_valid, y_valid_pred).ravel()
    Sn_collection.append(TP / (TP + FN))
    Sp_collection.append(TN / (TN + FP))
    ACC_collection.append((TP + TN) / (TP + TN + FP + FN))
    MCC = (TP * TN - FP * FN) / math.pow(((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)), 0.5)
    MCC_collection.append(MCC)
    BACC_collecton.append(0.5 * TP / (TP + FN) + 0.5 * TN / (TN + FP))

    # ROC-AUC
    auc_score = roc_auc_score(y_valid, y_valid_pred_proba[:, 1])
    AUC_collection.append(auc_score)

    # --- AP, AUPR, F1 ---
    ap = average_precision_score(y_valid, y_valid_pred_proba[:, 1])
    precision, recall, _ = precision_recall_curve(y_valid, y_valid_pred_proba[:, 1])
    aupr = auc(recall, precision)
    f1 = f1_score(y_valid, y_valid_pred)

    AP_collection.append(ap)
    AUPR_collection.append(aupr)
    F1_collection.append(f1)

# --- 保存预测结果 ---
results_df = pd.DataFrame({
    'Predicted_Proba': np.round(y_pred_proba_all, 8),
    'True_Label': y_true_all,
    'Predicted_Label': y_pred_all
})

# results_df.to_csv(r'E:/Experiment/work/DDE.csv', index=False)

# --- 输出评估结果 ---
# --- 输出评估结果 ---
print("ACC :", round(statistics.mean(ACC_collection), 3), '±', round(statistics.stdev(ACC_collection), 3))
print("Sn  :", round(statistics.mean(Sn_collection), 3), '±', round(statistics.stdev(Sn_collection), 3))
print("Sp  :", round(statistics.mean(Sp_collection), 3), '±', round(statistics.stdev(Sp_collection), 3))
print("MCC :", round(statistics.mean(MCC_collection), 3), '±', round(statistics.stdev(MCC_collection), 3))
print("BACC:", round(statistics.mean(BACC_collecton), 3), '±', round(statistics.stdev(BACC_collecton), 3))  # ⭐ 新增
print("AUC :", round(statistics.mean(AUC_collection), 3), '±', round(statistics.stdev(AUC_collection), 3))
print("AP  :", round(statistics.mean(AP_collection), 3), '±', round(statistics.stdev(AP_collection), 3))
print("AUPR:", round(statistics.mean(AUPR_collection), 3), '±', round(statistics.stdev(AUPR_collection), 3))
print("F1  :", round(statistics.mean(F1_collection), 3), '±', round(statistics.stdev(F1_collection), 3))


# # 绘制训练集和验证集的损失曲线
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss', color='blue', marker='o')
# plt.plot(range(1, len(valid_losses) + 1), valid_losses, label='Validation Loss', color='red', marker='x')
# plt.xlabel('Fold')
# plt.ylabel('Log Loss')
# plt.title('Training and Validation Loss Curve (Log Loss)')
# plt.legend()
# plt.grid(True)
# plt.show()


ACC : 0.608 ± 0.024
Sn  : 0.62 ± 0.013
Sp  : 0.577 ± 0.048
MCC : 0.177 ± 0.051
BACC: 0.599 ± 0.031
AUC : 0.616 ± 0.029
AP  : 0.542 ± 0.038
AUPR: 0.539 ± 0.038
F1  : 0.449 ± 0.019


### 独立测试

In [15]:
import math
import numpy as np
import statistics
import pandas as pd
from sklearn.metrics import (
    confusion_matrix, roc_auc_score, f1_score,
    average_precision_score, precision_recall_curve, auc
)

Sn_collection = []
AAC_collection = []
Sp_collection = []
MCC_collection = []
BACC_collection = []   # ⭐ 新增
AUC_collection = []
AP_collection = []
AUPR_collection = []
F1_collection = []
all_preds = []

def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)

for i in range(5):
    # 假设 clf、X_new, y_new, X_new1, y_new1 是预定义好的
    clf.fit(X_new, y_new)
    y_pred_score = clf.predict_proba(X_new1)
    y_pred = categorical_probas_to_classes(y_pred_score)
    y_true = y_new1

    # 混淆矩阵计算
    TP, FP, FN, TN = confusion_matrix(y_true, y_pred).ravel()
    Sn = TP / (TP + FN)
    Sp = TN / (TN + FP)
    ACC = (TP + TN) / (TP + TN + FP + FN)
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    BACC = 0.5 * (Sn + Sp)   # ⭐ 新增

    Sn_collection.append(Sn)
    AAC_collection.append(ACC)
    Sp_collection.append(Sp)
    MCC_collection.append(MCC)
    BACC_collection.append(BACC)   # ⭐ 保存BACC

    auc_score = roc_auc_score(y_true, y_pred_score[:, 1])
    AUC_collection.append(auc_score)

    # --- New: AP, AUPR and F1 ---
    ap = average_precision_score(y_true, y_pred_score[:, 1])
    precision, recall, _ = precision_recall_curve(y_true, y_pred_score[:, 1])
    aupr = auc(recall, precision)
    f1 = f1_score(y_true, y_pred)

    AP_collection.append(ap)
    AUPR_collection.append(aupr)
    F1_collection.append(f1)

    # 保存当前折的预测结果
    df_fold = pd.DataFrame({
        'Fold': [i + 1] * len(y_true),
        'True_Label': np.ravel(y_true),
        'Predicted_Label': np.ravel(y_pred),
        'Predicted_Probability': np.ravel(y_pred_score[:, 1])
    })
    all_preds.append(df_fold)

# 合并所有折结果
pred_results_df = pd.concat(all_preds, ignore_index=True)

# 保存到 CSV（按需开启）
# pred_results_df.to_csv(r'E:\Experiment\work\ESMC_test_LR.csv', index=False)

# --- 打印指标统计 ---
print("ACC :", round(statistics.mean(AAC_collection), 3), '±', round(statistics.stdev(AAC_collection), 3))
print("Sn  :", round(statistics.mean(Sn_collection), 3), '±', round(statistics.stdev(Sn_collection), 3))
print("Sp  :", round(statistics.mean(Sp_collection), 3), '±', round(statistics.stdev(Sp_collection), 3))
print("MCC :", round(statistics.mean(MCC_collection), 3), '±', round(statistics.stdev(MCC_collection), 3))
print("BACC:", round(statistics.mean(BACC_collection), 3), '±', round(statistics.stdev(BACC_collection), 3))  # ⭐ 新增
print("AUC :", round(statistics.mean(AUC_collection), 3), '±', round(statistics.stdev(AUC_collection), 3))
print("AP  :", round(statistics.mean(AP_collection), 3), '±', round(statistics.stdev(AP_collection), 3))
print("AUPR:", round(statistics.mean(AUPR_collection), 3), '±', round(statistics.stdev(AUPR_collection), 3))
print("F1  :", round(statistics.mean(F1_collection), 3), '±', round(statistics.stdev(F1_collection), 3))


ACC : 0.529 ± 0.0
Sn  : 0.52 ± 0.0
Sp  : 0.551 ± 0.0
MCC : 0.065 ± 0.0
BACC: 0.536 ± 0.0
AUC : 0.589 ± 0.0
AP  : 0.578 ± 0.0
AUPR: 0.575 ± 0.0
F1  : 0.401 ± 0.0


### 绘制AUPR曲线

In [77]:
import math
import numpy as np
import statistics
import pandas as pd
from sklearn.metrics import (
    confusion_matrix, roc_auc_score, f1_score,
    average_precision_score, precision_recall_curve, auc
)

Sn_collection = []
AAC_collection = []
Sp_collection = []
MCC_collection = []
AUC_collection = []
AP_collection = []
AUPR_collection = []
F1_collection = []
all_preds = []
pr_curves = []   # 保存PR曲线数据

def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)

for i in range(2):
    # 假设 clf、X_new, y_new, X_new1, y_new1 是预定义好的
    clf.fit(X_new, y_new)
    y_pred_score = clf.predict_proba(X_new1)
    y_pred = categorical_probas_to_classes(y_pred_score)
    y_true = y_new1

    # 混淆矩阵计算
    TP, FP, FN, TN = confusion_matrix(y_true, y_pred).ravel()
    Sn_collection.append(TP / (TP + FN))
    AAC_collection.append((TP + TN) / (TP + TN + FP + FN))
    Sp_collection.append(TN / (TN + FP))
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + TN))
    MCC_collection.append(MCC)

    auc_score = roc_auc_score(y_true, y_pred_score[:, 1])
    AUC_collection.append(auc_score)

    # --- New: AP, AUPR and F1 ---
    ap = average_precision_score(y_true, y_pred_score[:, 1])
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred_score[:, 1])
    aupr = auc(recall, precision)
    f1 = f1_score(y_true, y_pred)

    AP_collection.append(ap)
    AUPR_collection.append(aupr)
    F1_collection.append(f1)

    # 对齐 thresholds 长度：precision/recall 比 thresholds 多 1
    thresholds = np.append(thresholds, np.nan)

    # 保存 PR 曲线点 (Fold, Threshold, Precision, Recall)
    df_pr = pd.DataFrame({
        'Fold': [i + 1] * len(precision),
        'Threshold': thresholds,
        'Precision': precision,
        'Recall': recall
    })
    pr_curves.append(df_pr)

    # 保存当前折的预测结果（如果还需要）
    df_fold = pd.DataFrame({
        'Fold': [i + 1] * len(y_true),
        'True_Label': np.ravel(y_true),
        'Predicted_Label': np.ravel(y_pred),
        'Predicted_Probability': np.ravel(y_pred_score[:, 1])
    })
    all_preds.append(df_fold)

# 合并所有折结果
pred_results_df = pd.concat(all_preds, ignore_index=True)
pr_curves_df = pd.concat(pr_curves, ignore_index=True)

# 保存到 CSV
# pred_results_df.to_csv(r'G:\\lhy\work\AUPR\\test_LR_predictions.csv', index=False)
pr_curves_df.to_csv(r'G:\\lhy\work\AUPR\\{1}_test_LR_prcurve.csv', index=False)

# --- 打印指标统计 ---
print("ACC :", round(statistics.mean(AAC_collection), 3), '±', round(statistics.stdev(AAC_collection), 3))
print("Sn  :", round(statistics.mean(Sn_collection), 3), '±', round(statistics.stdev(Sn_collection), 3))
print("Sp  :", round(statistics.mean(Sp_collection), 3), '±', round(statistics.stdev(Sp_collection), 3))
print("MCC :", round(statistics.mean(MCC_collection), 3), '±', round(statistics.stdev(MCC_collection), 3))
print("AUC :", round(statistics.mean(AUC_collection), 3), '±', round(statistics.stdev(AUC_collection), 3))
print("AP  :", round(statistics.mean(AP_collection), 3), '±', round(statistics.stdev(AP_collection), 3))
print("AUPR:", round(statistics.mean(AUPR_collection), 3), '±', round(statistics.stdev(AUPR_collection), 3))
print("F1  :", round(statistics.mean(F1_collection), 3), '±', round(statistics.stdev(F1_collection), 3))


ACC : 0.529 ± 0.0
Sn  : 0.52 ± 0.0
Sp  : 0.551 ± 0.0
MCC : 0.081 ± 0.0
AUC : 0.589 ± 0.0
AP  : 0.578 ± 0.0
AUPR: 0.575 ± 0.0
F1  : 0.401 ± 0.0


### AUROC

In [ ]:
import pandas as pd
import numpy as np

all_preds = []

for i in range(2):
    clf.fit(X_new, y_new)
    y_pred_score = clf.predict_proba(X_new1)
    y_pred = np.argmax(y_pred_score, axis=1)
    y_true = y_new1

    df_fold = pd.DataFrame({
        'Fold': [i + 1] * len(y_true),
        'Predicted_Proba': y_pred_score[:, 1].ravel(),
        'True_Label': np.ravel(y_true),
        'Predicted_Label': y_pred.ravel()    
    })
    all_preds.append(df_fold)

pred_results_df = pd.concat(all_preds, ignore_index=True)
# pred_results_df.to_csv(r'G:\lhy\work\AUPR\test_LR_AUROC.csv', index=False)


# 保存到 CSV
pred_results_df.to_csv(r'G:\\lhy\work\AUROC\\{1,2,3,4,5}_test_LR_AUROC.csv', index=False)


In [ ]:
# Independence test
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# result collection list
BACC_collecton = []
ACC_collection = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
AP=[]
mean_recall = np.linspace(0, 1, 100)
all_precision = []
base_fpr = np.linspace(0, 1, 100)
mean_tpr = 0.0
# 新的TPR集合
interp_tpr_collection = []

def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)

for i in range(10):
    # dataset splitting
    X_train_whole, X_ind_test, y_train_whole, y_ind_test = train_test_split(X_new, y_new, test_size=0.2, random_state=i)
    clf = LogisticRegression(C = 17.47528400007683, max_iter=5000)
    clf.fit(X_train_whole, y_train_whole)   
    y_pred_score = clf.predict_proba(X_new1)
    y_pred = categorical_probas_to_classes(y_pred_score) 
    print(y_pred)   
    y_true = y_ind_test